# Build vitessce app for CODEX annotations

- Need to include explicit instructions on how to view app

In [1]:
import tifffile as tff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import anndata as ad
import pickle as pk
import scanpy as sc
from vitessce.data_utils import (
    multiplex_img_to_ome_zarr,
    optimize_adata,
    VAR_CHUNK_SIZE,
)
from vitessce import (
    VitessceConfig,
    VitessceChainableConfig, 
    VitessceConfigDatasetFile,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
    OmeTiffWrapper,
    OmeZarrWrapper,
    CsvWrapper,
    MultiImageWrapper,
    AbstractWrapper,
)
import json
from urllib.parse import quote_plus

In [15]:
adata = ad.read_h5ad("/mnt/isilon/cscb/Projects/HubMAP/processed/seurat/CODEX/immune.combined_final_040523.h5ad")
adata

/mnt/isilon/cscb/software/anaconda3/envs/vitessce/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 886525 × 49
    obs: 'nCount_CODEX', 'nFeature_CODEX', 'orig.ident', 'CellID', 'Size', 'x.coord', 'y.coord', 'DAPI', 'Metadata_CD19', 'Metadata_ASMA', 'Metadata_CD4', 'Metadata_CD44', 'Metadata_CD71', 'Metadata_CD271', 'Metadata_CD90', 'Metadata_VIM', 'Metadata_CD68', 'Metadata_SPINK2', 'Metadata_CD117', 'Metadata_CD45RA', 'Metadata_FOXC1', 'Metadata_CD61', 'Metadata_CD45', 'Metadata_CD8', 'Metadata_CD49F', 'Metadata_CD11C', 'Metadata_VECAD', 'Metadata_CD15', 'ADIPOQ', 'Metadata_CD123', 'Metadata_CD138', 'Metadata_CD14', 'Metadata_PLP1', 'Metadata_CD11B', 'Metadata_HLA.DR', 'Metadata_CD34', 'Metadata_CD79A', 'Metadata_CD10', 'Metadata_PAX5', 'Metadata_MastCellTryptase', 'Metadata_TGFB1', 'Metadata_VCAM1', 'Metadata_CD146', 'Ki67', 'Metadata_CD3e', 'Metadata_CD33', 'Metadata_GATA1', 'CD56', 'Metadata_CXCL12', 'NAKATPASE', 'Metadata_BCL2', 'Metadata_GYPC', 'Metadata_HIF1A', 'Metadata_MPO', 'Metadata_OXPHOS', 'Metadata_CD38', 'Metadata_CD141', 'Metadat

In [25]:
adata.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [8]:
# del(adata.var['_index'])

In [21]:
adata.var_names = adata.__dict__['_raw'].__dict__['_var']['_index']
# adata.__dict__['_raw'].__dict__['_var']['_index']

In [22]:
out = adata.copy()
out.obs = out.obs.loc[:,['orig.ident', 'nCount_CODEX', 'nFeature_CODEX','cluster_anno_l1', 'cluster_anno_l2', 'cluster_anno_coarse']]
out.__dict__['_raw'].__dict__['_var'] = out.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [30]:
out.__dict__['_var'].drop(columns=["_index"], inplace=True)

In [31]:
out.write("/mnt/isilon/cscb/Projects/HubMAP/processed/seurat/CODEX/immune.combined_forCellxGene.h5ad")

In [2]:
adata = ad.read_h5ad("/mnt/isilon/cscb/Projects/HubMAP/processed/seurat/CODEX/immune.combined_forCellxGene.h5ad")

In [3]:
zarr_filepath = "/mnt/isilon/cscb/Projects/HubMAP/processed/seurat/CODEX/vite_adata.zarr"

In [6]:
adata_out = optimize_adata(
    adata,
    obs_cols=["orig.ident","cluster_anno_l1","cluster_anno_l2","cluster_anno_coarse"],
    obsm_keys=["X_umap", "X_UMAP_dim30"],
    optimize_X=True,
)
adata_out.write_zarr(zarr_filepath, chunks=[adata_out.shape[0], VAR_CHUNK_SIZE])

In [15]:
# just getting the local server running
vc = VitessceConfig(schema_version="1.0.15", name='CODEX', description='Testing small slice of CODEX image')
# "obs..." stuff shouldn't matter...
dataset = vc.add_dataset(name='R1reg1_obs').add_object(AnnDataWrapper(
    adata_path=zarr_filepath,
    obs_set_paths=["obs/new_pheno"],
    obs_set_names=["phenotype"],
    obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
    obs_embedding_names=["UMAP", "PCA"],
    obs_feature_matrix_path="X"
))
lc = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset)
umap = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)


In [17]:
vw = vc.widget()
print(vw)

VitessceWidget(config={'version': '1.0.15', 'name': 'CODEX', 'description': 'Testing small slice of CODEX image', 'datasets': [{'uid': 'A', 'name': 'R1reg1_obs', 'files': [{'fileType': 'anndata.zarr', 'url': 'http://localhost:8000/A/0/3fb0b613-269d-4f7c-a971-b2c29d0cda52', 'options': {'obsEmbedding': [{'path': 'obsm/X_umap', 'dims': [0, 1], 'embeddingType': 'UMAP'}, {'path': 'obsm/X_pca', 'dims': [0, 1], 'embeddingType': 'PCA'}], 'obsSets': [{'name': 'phenotype', 'path': 'obs/new_pheno'}], 'obsFeatureMatrix': {'path': 'X'}}}]}], 'coordinationSpace': {'dataset': {'A': 'A'}, 'embeddingType': {'A': 'UMAP'}}, 'layout': [{'component': 'layerController', 'coordinationScopes': {'dataset': 'A'}, 'x': 0, 'y': 0, 'w': 1, 'h': 1}, {'component': 'scatterplot', 'coordinationScopes': {'dataset': 'A', 'embeddingType': 'A'}, 'x': 0, 'y': 0, 'w': 1, 'h': 1}, {'component': 'obsSets', 'coordinationScopes': {'dataset': 'A'}, 'x': 0, 'y': 0, 'w': 1, 'h': 1}, {'component': 'featureList', 'coordinationScopes

In [21]:
vc.__dict__.keys()

dict_keys(['config', 'background_servers', 'base_dir'])

In [17]:
# HuBMAP style
conf = VitessceChainableConfig(
    name="R1_reg1_expr", description="", schema_version="1.0.7" # upgraded schema from 1.0.7
).add_dataset(
    uid="A",
    name="R1_reg1",
    files=[
        VitessceConfigDatasetFile(
            # data_type="cells",
            file_type="anndata-cells.zarr",
            # url="http://localhost:8000/A/0/3fb0b613-269d-4f7c-a971-b2c29d0cda52",
            url="https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687",
            options={
                "mappings": {"UMAP": {"key": "obsm/X_umap", "dims": [0, 1]}}, # this might be how I apply segmentations with JSON, then set_coordination_value()...
                "factors": [
                    "obs/orig.ident",
                    "obs/cluster_anno_coarse",
                    "obs/cluster_anno_l1",
                    "obs/cluster_anno_l2",
                ],
            },
        ),
        VitessceConfigDatasetFile(
            # data_type="cell-sets",
            file_type="anndata-cell-sets.zarr",
            url="https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687",
            options=[
                {
                    "groupName": "origin",
                    "setName": "obs/orig.ident",
                },
                {
                    "groupName": "celltype_coarse",
                    "setName": "obs/cluster_anno_coarse",
                },
                {
                    "groupName": "celltype_l1",
                    "setName": "obs/cluster_anno_l1",
                },
                {
                    "groupName": "celltype_l2",
                    "setName": "obs/cluster_anno_l2",
                },
            ],
        ),
        VitessceConfigDatasetFile(
            # data_type="expression-matrix",
            file_type="anndata-expression-matrix.zarr",
            url="https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687",
            options={"matrix": "X"},
        ),
    ],
).set_coordination_value(
    c_type="embeddingType", c_scope="A", c_value="UMAP"
).add_view(
    dataset_uid="A", component="description", x=0, y=8, w=3, h=4
).add_view(
    dataset_uid="A",
    component="scatterplot",
    x=0,
    y=0,
    w=6,
    h=6,
    coordination_scopes={"embeddingType": "A"},
).add_view(
    dataset_uid="A", component="cellSets", x=6, y=5, w=2, h=7
).add_view(
    dataset_uid="A",
    component="genes",
    x=6,
    y=0,
    w=2,
    h=5,
    props={"variablesLabelOverride": "antigen"},
).add_view(
    dataset_uid="A",
    component="heatmap",
    x=0,
    y=6,
    w=6,
    h=4,
    props={"variablesLabelOverride": "antigen", "transpose": True},
)

# conf.widget()

In [19]:
vw = conf.widget()
print(vw)

VitessceWidget(config={'version': '1.0.7', 'name': 'R1_reg1_expr', 'description': '', 'datasets': [{'uid': 'A', 'name': 'R1_reg1', 'files': [{'fileType': 'anndata-cells.zarr', 'url': 'https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687', 'options': {'mappings': {'UMAP': {'key': 'obsm/X_umap', 'dims': [0, 1]}}, 'factors': ['obs/orig.ident', 'obs/cluster_anno_coarse', 'obs/cluster_anno_l1', 'obs/cluster_anno_l2']}}, {'fileType': 'anndata-cell-sets.zarr', 'url': 'https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687', 'options': [{'groupName': 'origin', 'setName': 'obs/orig.ident'}, {'groupName': 'celltype_coarse', 'setName': 'obs/cluster_anno_coarse'}, {'groupName': 'celltype_l1', 'setName': 'obs/cluster_anno_l1'}, {'groupName': 'celltype_l2', 'setName': 'obs/cluster_anno_l2'}]}, {'fileType': 'anndata-expression-matrix.zarr', 'url': 'https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd

In [20]:
config_dict = conf.export(to='files', base_url='https://cscb.research.chop.edu/share_data/HuBMAP', out_dir='.')

In [21]:
config_dict

{'version': '1.0.7',
 'name': 'R1_reg1_expr',
 'description': '',
 'datasets': [{'uid': 'A',
   'name': 'R1_reg1',
   'files': [{'fileType': 'anndata-cells.zarr',
     'url': 'https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687',
     'options': {'mappings': {'UMAP': {'key': 'obsm/X_umap', 'dims': [0, 1]}},
      'factors': ['obs/orig.ident',
       'obs/cluster_anno_coarse',
       'obs/cluster_anno_l1',
       'obs/cluster_anno_l2']}},
    {'fileType': 'anndata-cell-sets.zarr',
     'url': 'https://cscb.research.chop.edu/share_data/HuBMAP/A/0/5e3a83c0-4f0c-4263-84ba-fa2dd3e10687',
     'options': [{'groupName': 'origin', 'setName': 'obs/orig.ident'},
      {'groupName': 'celltype_coarse', 'setName': 'obs/cluster_anno_coarse'},
      {'groupName': 'celltype_l1', 'setName': 'obs/cluster_anno_l1'},
      {'groupName': 'celltype_l2', 'setName': 'obs/cluster_anno_l2'}]},
    {'fileType': 'anndata-expression-matrix.zarr',
     'url': 'https://cscb.rese

In [16]:
vitessce_url = "http://vitessce.io/?url=data:," + quote_plus(json.dumps(config_dict))
vitessce_url

'http://vitessce.io/?url=data:,%7B%22version%22%3A+%221.0.7%22%2C+%22name%22%3A+%22R1_reg1_expr%22%2C+%22description%22%3A+%22%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22R1_reg1%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22anndata-cells.zarr%22%2C+%22url%22%3A+%22https%3A%2F%2Fcscb.research.chop.edu%2Fshare_data%2FHuBMAP%2FA%2F0%2F3fb0b613-269d-4f7c-a971-b2c29d0cda52%22%2C+%22options%22%3A+%7B%22mappings%22%3A+%7B%22UMAP%22%3A+%7B%22key%22%3A+%22obsm%2FX_umap%22%2C+%22dims%22%3A+%5B0%2C+1%5D%7D%7D%2C+%22factors%22%3A+%5B%22obs%2Forig.ident%22%2C+%22obs%2Fcluster_anno_coarse%22%2C+%22obs%2Fcluster_anno_l1%22%2C+%22obs%2Fcluster_anno_l2%22%5D%7D%7D%2C+%7B%22fileType%22%3A+%22anndata-cell-sets.zarr%22%2C+%22url%22%3A+%22https%3A%2F%2Fcscb.research.chop.edu%2Fshare_data%2FHuBMAP%2FA%2F0%2F3fb0b613-269d-4f7c-a971-b2c29d0cda52%22%2C+%22options%22%3A+%5B%7B%22groupName%22%3A+%22origin%22%2C+%22setName%22%3A+%22obs%2Forig.ident%22%7D%2C+%7B%22groupName%22%3A+%22c

In [26]:
conf.display()